In [1]:
import os
import glob
import codecs
import spacy
import json
import pandas as pd
import numpy as np
import networkx as nx

import nltk
from nltk.tokenize import sent_tokenize
import spacy
from spacy import displacy
from collections import Counter

from pathlib import Path
from pyspark import SparkContext
from afinn import Afinn
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [3]:
import re

In [4]:
def flatten(input_list):
    '''
    A function to flatten complex list.
    :param input_list: The list to be flatten
    :return: the flattened list.
    '''

    flat_list = []
    for i in input_list:
        if type(i) == list:
            flat_list += flatten(i)
        else:
            flat_list += [i]

    return flat_list


def name_entity_recognition(sentence,names_list):
    '''
    A function to retrieve name entities in a sentence.
    :param sentence: the sentence to retrieve names from.
    :return: a name entity list of the sentence.
    '''

    doc = nlp(sentence)
    # retrieve person and organization's name from the sentence
    name_entity = [x for x in doc.ents if x.label_ in ['PERSON', 'ORG']]
    # convert all names to lowercase and remove 's in names
    name_entity = [str(x).lower().replace("'s","") for x in name_entity]
    # split names into single words ('Harry Potter' -> ['Harry', 'Potter'])
    name_entity = [x.split(' ') for x in name_entity]
    # flatten the name list
    name_entity = flatten(name_entity)
    # remove name words that are less than 3 letters to raise recognition accuracy
    name_entity = [x for x in name_entity if len(x) >= 3]
    # remove name words that are in the set of 4000 common words
    name_entity = [x for x in name_entity if x not in common_words]
    # remove name words that are in the set of races
    name_entity = [x for x in name_entity if x not in races]
    #print(name_entity)
    for name in name_entity:
        names_list.append(name)
    return name_entity

In [5]:
sent_file = open(r'C:\Users\NYashch\nyashch-projects\all_sentences.txt','r',encoding = 'utf-8')
all_sentences = [line.strip() for line in sent_file]
sent_file.close()
tolkien_text = ''.join(all_sentences)

In [6]:
'''
tolkien_text_new = []
for sent in tolkien_text.split('.'):
    list_sent = sent.split()
    if 'Mithrandir' in list_sent:
        new_sent = sent.replace('Mithrandir', 'gendalf')
        tolkien_text_new.append(new_sent)
    elif 'Strider' in list_sent:
        new_sent = sent.replace('Strider', 'aragorn')
        tolkien_text_new.append(new_sent)
    elif 'Gamgee' in list_sent:
        new_sent = sent.replace('Gamgee', 'sam')
        tolkien_text_new.append(new_sent)
    elif 'Meriadoc' in list_sent:
        new_sent = sent.replace('Meriadoc', 'merry')
        tolkien_text_new.append(new_sent)
    elif 'Peregrin' in list_sent:
        new_sent = sent.replace('Peregrin', 'pippin')
        tolkien_text_new.append(new_sent)
    else:
        tolkien_text_new.append(new_sent)
'''

"\ntolkien_text_new = []\nfor sent in tolkien_text.split('.'):\n    list_sent = sent.split()\n    if 'Mithrandir' in list_sent:\n        new_sent = sent.replace('Mithrandir', 'gendalf')\n        tolkien_text_new.append(new_sent)\n    elif 'Strider' in list_sent:\n        new_sent = sent.replace('Strider', 'aragorn')\n        tolkien_text_new.append(new_sent)\n    elif 'Gamgee' in list_sent:\n        new_sent = sent.replace('Gamgee', 'sam')\n        tolkien_text_new.append(new_sent)\n    elif 'Meriadoc' in list_sent:\n        new_sent = sent.replace('Meriadoc', 'merry')\n        tolkien_text_new.append(new_sent)\n    elif 'Peregrin' in list_sent:\n        new_sent = sent.replace('Peregrin', 'pippin')\n        tolkien_text_new.append(new_sent)\n    else:\n        tolkien_text_new.append(new_sent)\n"

In [7]:
'''names = []
f = open(r'C:\Users\NYashch\nyashch-projects\my_names.txt','r',encoding = 'utf-8')
for elem in f.readlines():
    elem = elem.replace("\n","")
    names.append(elem)
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 24-25: truncated \UXXXXXXXX escape (<ipython-input-7-bf55be2255d0>, line 1)

In [6]:
import re 
my_hobbits = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_hobbits.txt','r',encoding = 'utf-8')
hobbits = []
for elem in my_hobbits.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        hobbits.append(elem.lower().split()[0])

d_hobbits = dict.fromkeys(hobbits, 'hobbit')

my_dwarves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_dwarves.txt','r',encoding = 'utf-8')
dwarves = []

for elem in my_dwarves.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        dwarves.append(elem.lower().split()[0])
d_dwarves = dict.fromkeys(dwarves, 'dwarf')
my_elves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_elves.txt','r',encoding = 'utf-8')
elves = []
for elem in my_elves .readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        elves.append(elem.lower().split()[0])
d_elves = dict.fromkeys(elves, 'elf')
my_maiar = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_maiar.txt','r',encoding = 'utf-8')
maiar = []
for elem in my_maiar.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        maiar.append(elem.lower().split()[0])
d_maiar = dict.fromkeys(maiar, 'maiar')
my_orcs = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_orcs.txt','r',encoding = 'utf-8')
orcs = []
for elem in my_orcs.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        orcs.append(elem.lower().split()[0])
d_orcs = dict.fromkeys(orcs, 'orc')
my_men = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_men.txt','r',encoding = 'utf-8')
men = []
for elem in my_men.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        men.append(elem.lower().split()[0])
d_men = dict.fromkeys(men, 'men')
my_valars = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_valars.txt','r',encoding = 'utf-8')
valars = []
for elem in my_valars.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        valars.append(elem.lower())
d_valars = dict.fromkeys(valars, 'valar')
ainur = ["ainur",'aluin','danuin','fanuin','morgoth','ranuin']
d_ainur = dict.fromkeys(ainur, 'ainur')
barlogs = ['gothmog','lungorthin','osombauko']
d_barlogs = dict.fromkeys(barlogs, 'barlog')
dragons = ['ancalagon','chrysophylax','glaurung','gostir','scatha','smaug','urulókë']
d_dragons = dict.fromkeys(dragons, 'dragon')
eagles = ['gwaihir','landroval','meneldor','thornhoth','thorondor','windlord']
d_eagles = dict.fromkeys(eagles, 'eagle')
ents = ['beechbone','entings','entmaidens','entmoot','entwives','fimbrethil','leaflock','onodrim','quickbeam','skinbark','treebeard']
d_ents = dict.fromkeys(ents, 'ent')
half_elven = ['ardamir eärendil','arwen','dior','eärendil','elladan','elrohir','elrond','elros','elurín','elwing','undómiel']
d_half_elven = dict.fromkeys(half_elven, 'half_elf')
werewolves = ['carcharoth','draugluin','thû']
d_werewolves = dict.fromkeys(werewolves, 'werewolf')
trolls = ['bert','olog-hai','tom','william']
d_trolls = dict.fromkeys(trolls, 'troll')


In [7]:
all_names = hobbits + dwarves +elves+maiar+orcs+men+valars+ainur+barlogs+dragons+eagles+ents+half_elven+werewolves+trolls

In [8]:
names = list(set(all_names))

In [9]:
extra = ['big', 'elven']

In [10]:
for elem in extra:
        if elem in names:
            names.remove(elem)


In [11]:

names.append('sam')

In [12]:
name_race = {}
for name in names:
    if name in d_hobbits:
        name_race[name] ='hobbit'
    elif name in d_elves:
        name_race[name] ='elf'
    elif name in  d_dwarves:
        name_race[name] ='dwarf'
    elif name in  d_trolls:
        name_race[name] ='troll'
    elif name in  d_werewolves:
        name_race[name] ='werewolf'
    elif name in  d_ents:
        name_race[name] ='ent'
    elif name in  d_half_elven:
        name_race[name] ='elf'
    elif name in  d_eagles:
        name_race[name] ='eagle'
    elif name in  d_dragons:
        name_race[name] ='dragon'
    elif name in  d_barlogs:
        name_race[name] ='barlog'
    elif name in  d_ainur:
        name_race[name] ='ainur'
    elif name in  d_valars:
        name_race[name] ='valar'
    elif name in  d_orcs:
        name_race[name] ='orc'
    elif name in  d_men:
        name_race[name] ='man'
    elif name in  d_maiar:
        name_race[name] ='maiar'
    elif name == 'manwë':
        name_race[name] ='valar'
    elif name == 'sam':
        name_race[name] ='hobbit'
    elif name == 'melkor':
        name_race[name] ='maiar'
    elif name =='pippin':
        name_race[name] ='hobbit'
    elif name == 'arda':
        name_race[name] ='arda'

    else:
        names.remove(name)
 


In [30]:
race = []
for name in names:
    race.append(name_race[name])
race = race[:140]

In [14]:
nr_data = {"name": names, 'race': race}
name_race_df = pd.DataFrame.from_dict(nr_data)
name_race_df.to_csv('name_race_df.csv')

In [15]:
print(len(list(set(race))))

14


In [16]:
'''from afinn import Afinn
afinn = Afinn()
d = {}
counter = 0
for sent in ner_sentences:
    counter += 1
    if counter % 1000 == 0:
        print(counter)
    sent_names = []
    for name in names:
        if name in sent.lower().split():
            sent_names.append(name)
    #name_entity_recognition(sent,sent_names)
    if len(sent_names) > 1:
        for name1 in sent_names:
            for name2 in sent_names:
                if name1 != name2:
                    key = ""
                    if name1 < name2:
                        key = name1 + "#" + name2
                    else:
                        key = name2 + "#" + name1
                    if d.get(key) == None:
                        d[key] =afinn.score(sent)
                    else:
                        d[key] = d[key]+afinn.score(sent)
'''

NameError: name 'ner_sentences' is not defined

In [ ]:
'''items_list= []
for i in d.items():
    l = []
    line = list(i)[0].split('#')
    for elem in line:
        l.append(elem)
    l.append(list(i)[1])
    items_list.append(l)
sentiment_counter = -1
relations = []
for i in items_list:
    if i[2] < -5:
        relations.append('enemy')
    if i[2] > 2:
        relations.append('friend')
    else:
        relations.append('neutral')
names1 = []
names2 = []
edge_colors = []

for i in items_list:
    sentiment_counter += 1
    names1.append(i[0])
    names2.append(i[1])
    edge_colors.append(relations[sentiment_counter])
label_colors = []
for name in names1:
        label_colors.append(color_map[name])
data = {'name1': names1,'name2': names2,'label_color': label_colors,'edge_color': edge_colors}

relations_data = pd.DataFrame.from_dict(data)
'''

In [ ]:
relations_data.to_csv('relations_data_df.csv')

EDGE BETWENESS


In [32]:
eb_df = pd.read_csv(r'C:\Users\NYashch\nyashch-projects\Diploma\eb_df.csv')

In [39]:
eb = list(eb_df['members'])


In [ ]:
'''
true_clusters = []
for name in cluster_names:
    if name in names:
        true_clusters.append(name_race[name])
    elif name == 'feanor':
        true_clusters.append('elf')
    elif name == 'luthien':
        true_clusters.append('elf')
    elif name == 'hurin':
        true_clusters.append('man')
    elif name == 'earendil':
        true_clusters.append('elf')
    elif name == 'elurin':
        true_clusters.append('elf')
    elif name == 'theoden':
        true_clusters.append('man')
    elif name == 'eomer':
        true_clusters.append('elf')
    elif name == 'eowyn':
        true_clusters.append('elf')
    elif name == 'undomiel':
        true_clusters.append('elf')
'''

GREEDY MODULARITY

In [23]:
mm_df = pd.read_csv(r'C:\Users\NYashch\nyashch-projects\Diploma\mm_df.csv')

In [27]:
mm = list(mm_df['members'])
print(len(set(mm)))

7


LABEL PROPOGATION

In [34]:
lp_df = pd.read_csv(r'C:\Users\NYashch\nyashch-projects\Diploma\lp_df.csv')

In [35]:
lp = list(lp_df['members'])
print(len(set(lp)))

4


In [28]:
from sklearn.metrics.cluster import adjusted_rand_score

In [36]:
adjusted_rand_score(race,mm)

0.003196803752697034

In [40]:
adjusted_rand_score(race,eb)

0.011915830774922857

In [38]:
adjusted_rand_score(race,lp)

0.0035420700277588747